In [0]:
USE CATALOG claims360_dev;
USE SCHEMA silver;

-- map denial reasons

CREATE TABLE IF NOT EXISTS silver.dim_denial_reason_map (
  code_type       STRING,          -- 'CARC' or 'RARC'
  code            STRING,          -- e.g. 'CO45'
  reason_category STRING,          -- normalized bucket
  is_denial       BOOLEAN,         -- true if this should count as a denial
  notes           STRING
) USING DELTA;

INSERT OVERWRITE TABLE silver.dim_denial_reason_map VALUES
-- Pricing/contractual
('CARC','CO45','Pricing/Contractual', true,  'Charge exceeds fee schedule/maximum'),
('CARC','CO97','Bundled/Included',    true,  'Included in allowance for another service'),

-- Coordination of Benefits / prior payer
('CARC','OA23','Coordination of Benefits', true, 'Impact of prior payer adjudication'),

-- Timely filing
('CARC','CO29','Timely Filing',        true, 'Time limit for filing expired'),

-- Patient responsibility
('CARC','PR1', 'Patient Responsibility', true, 'Deductible amount'),

-- Payer policy/clinical edit (PI = Payer Initiated)
('CARC','PI204','Policy/Clinical Edit', true, 'Service included in another procedure'),

-- Catchalls to refine over time
('CARC','CO',   'Contractual/Other',    true,  'Other CO family'),
('CARC','PR',   'Patient Responsibility',true, 'Other PR family'),
('CARC','PI',   'Policy/Clinical Edit', true,  'Other PI family'),
('CARC','OA',   'Other Adjustment',     true,  'Other OA family'),

-- RARC rows (none in data)
('RARC','N130','Documentation',         true,  'Incomplete/invalid documentation');

In [0]:
USE CATALOG claims360_dev;
USE SCHEMA silver;

-- Dimension table
CREATE OR REPLACE TABLE silver.dim_payer (
  payer_id   STRING NOT NULL,
  payer_name STRING NOT NULL,
  payer_type STRING NOT NULL,
  updated_at TIMESTAMP
)
USING DELTA;

-- Inline data as a temp view (session-scoped)
CREATE OR REPLACE TEMP VIEW _dim_payer_upsert AS
SELECT *
FROM VALUES
  ('PAYER001', 'Aetna',            'Commercial'),
  ('PAYER002', 'Anthem BCBS',      'Commercial'),
  ('PAYER003', 'Cigna',            'Commercial'),
  ('PAYER004', 'UnitedHealthcare', 'Commercial'),
  ('PAYER005', 'Humana',           'Commercial'),
  ('PAYER006', 'Kaiser',           'Commercial'),
  ('PAYER007', 'Medicare',         'Government'),
  ('PAYER008', 'Medicaid',         'Government')  
AS t(payer_id, payer_name, payer_type);

-- Merge (idempotent)
MERGE INTO silver.dim_payer d
USING (
  SELECT payer_id, payer_name, payer_type
  FROM _dim_payer_upsert
) s
ON d.payer_id = s.payer_id
WHEN MATCHED THEN
  UPDATE SET
    d.payer_name = s.payer_name,
    d.payer_type = s.payer_type,
    d.updated_at = current_timestamp()
WHEN NOT MATCHED THEN
  INSERT (payer_id, payer_name, payer_type)
  VALUES (s.payer_id, s.payer_name, s.payer_type);